In [27]:
from typing import Iterable, Mapping, Callable, TypeVar, Generic, MutableMapping, Sequence
import os
from haskellian import either as E
from moveread.core import CoreAPI, Game
from moveread.annotations import GameMeta, Tournament
import chess_utils as cu
import chess_pairings as cp
T = TypeVar('T')

In [2]:
all_games = list(cu.read_pgns(open('/mnt/j/My Drive/original/llobregat23/livegames/all.pgn')))
games = cp.classify(all_games, headers2group=lambda _: 'a', headers2tournId=lambda _: 'llobregat23')

illegal san: 'Kd5' in 3r2r1/4Q3/4p3/1R2kp2/8/6P1/2q2P1P/3R2K1 b - - 2 35 while parsing <Game at 0x7f7e98fb8090 ('Garcia Ramos, Daniel' vs. 'Birkisson, Bardur Orn', '2023.11.30' at 'Sant Boi de Llobregat, Spain')>
illegal san: 'Kxe5' in 4r3/5p2/2p1k1p1/1p1pPnPp/p2P3P/P1P2P1B/1R2K3/8 b - - 1 39 while parsing <Game at 0x7f7e98563490 ('Sorensen, Hampus' vs. 'Hulka, Valentyn', '2023.12.03' at 'Sant Boi de Llobregat, Spain')>
illegal san: 'Kd4' in 3r4/1p6/2p1k3/2P1pp1p/P1R2n2/4KP2/2Br4/6R1 w - - 1 45 while parsing <Game at 0x7f7e97493710 ('Rakshitta, Ravi' vs. 'Narciso Dublan, Marc', '2023.12.08' at 'Sant Boi de Llobregat, Spain')>


In [29]:
core = CoreAPI.at('/home/m4rs/.core/llobregat23')
core_games = await core.games.values().map(E.unsafe).sync()

In [30]:
def parse(id: str) -> cp.GameId:
  grp, rnd, brd = id.split('/')
  return cp.gameId('llobregat23', grp, rnd, brd)

def dump(tournId: str, group: str, round: str, board: str) -> str:
  return f'{group}/{round}/{board}'

core_index = cp.GamesMapping[Game].from_pairs([(parse(g.id), g) for g in core_games])

In [31]:
from moveread.core.models import Game

for gid in games.gameIds():
  if gid in core_index:
    game = core_index[gid]
    meta: GameMeta = game.meta # type: ignore
    dirty = False
    if meta.pgn is None:
      meta.pgn = cu.read_sans(games[gid])
      dirty = True
    if meta.tournament is None:
      meta.tournament = Tournament(**gid)
      dirty = True
    elif meta.tournament != Tournament(**gid):
      print(f'Oops! {gid}')

    if dirty:
      print(game.id)
      (await core.games.insert(game.id, game)).unsafe()

a/1/1


In [32]:
for i, game in enumerate(core_games):
  # print(f'\r{i+1}/{len(core_games)}', end='', flush=True)
  gid = parse(game.id)
  meta = game.meta
  if (meta is None or meta.pgn is None) and gid in games:
    print(game.id)